In [5]:
%%configure -f
{"conf": {"spark.jars": "/jar/simpleHTM.jar"}}

Starting Spark application


In [5]:
import simple.HTMindex
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
38,application_1568670927228_0041,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


<console>:23: error: not found: value simple
       import simple.HTMindex
              ^



In [13]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

read object table parquet files into dataframe (objDF)

create new dataframe (newDF) with calculated htmid

In [17]:
val datafile = "/user/hive/warehouse/object_test_parquet"
val objDF = spark.read.parquet(datafile)

val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl")))
newDF.printSchema()
newDF.show(1)

write newDF to external table in SqlDataPool using Spark-SQL connector


In [0]:
val start = Calendar.getInstance().getTime()
val end = Calendar.getInstance().getTime()



In [0]:

println("Use MSSQL connector to write to master SQL instance ")

val servername = "jdbc:sqlserver://master-0.master-svc"
val dbname = "LSST"
var url = servername + ";" + "databaseName=" + dbname + ";"

val user = "sa"
val password = "fooRiuzg54" # Please specify password here

val datapool_table = "ObjectDP_test_htmid"
val datasource_name = "SqlDataPool"

val batchsize = 1000000

val start = Calendar.getInstance().getTime()

try {
  newDF.write 
    .format("com.microsoft.sqlserver.jdbc.spark") 
    .mode("append") 
    .option("url", url) 
    .option("dbtable", datapool_table) 
    .option("user", user) 
    .option("password", password) 
    .option("dataPoolDataSource",datasource_name)
    .option("batchsize",batchsize)
    .save()
} catch {
    case e: Throwable => println("MSSQL Connector write failed: " + e)
}
val end = Calendar.getInstance().getTime()
var duration = end - start
println("duration:" + duration)
print("MSSQL Connector write(overwrite) to data pool external table succeeded")